In [9]:
from functools import partial

import sklearn
import sklearn.datasets
import sklearn.ensemble
import multiprocessing as mp
import numpy as np
import lime
import lime.lime_tabular
np.random.seed(1)

### Loading data, training a model

For this part, we'll use the Iris dataset, and we'll train a random forest. 

In [2]:
iris = sklearn.datasets.load_iris()

In [3]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris.data, iris.target, train_size=0.80)


In [4]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train)


RandomForestClassifier(n_estimators=500)

In [5]:
sklearn.metrics.accuracy_score(labels_test, rf.predict(test))

0.9666666666666667

### Create the explainer

In [6]:
explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=iris.feature_names, class_names=iris.target_names, discretize_continuous=True)

### Explaining multiple instances in parallel

In [11]:
%%time
explainer_partial = partial(
    explainer.explain_instance, predict_fn=rf.predict_proba, num_features=2
)
with mp.Pool(mp.cpu_count() - 1) as p:
    exp_parallel = p.map(explainer_partial, test[:20])

print(exp_parallel)

[<lime.explanation.Explanation object at 0x7fc166148190>, <lime.explanation.Explanation object at 0x7fc166148710>, <lime.explanation.Explanation object at 0x7fc166034690>, <lime.explanation.Explanation object at 0x7fc166148650>, <lime.explanation.Explanation object at 0x7fc165aa8b90>, <lime.explanation.Explanation object at 0x7fc166148d10>, <lime.explanation.Explanation object at 0x7fc16610f810>, <lime.explanation.Explanation object at 0x7fc166148050>, <lime.explanation.Explanation object at 0x7fc166197510>, <lime.explanation.Explanation object at 0x7fc166197610>, <lime.explanation.Explanation object at 0x7fc166197d90>, <lime.explanation.Explanation object at 0x7fc15de91a90>, <lime.explanation.Explanation object at 0x7fc15de91890>, <lime.explanation.Explanation object at 0x7fc15de91790>, <lime.explanation.Explanation object at 0x7fc15de91bd0>, <lime.explanation.Explanation object at 0x7fc166191e50>, <lime.explanation.Explanation object at 0x7fc166191e90>, <lime.explanation.Explanation 

In [18]:
exp_parallel[0].show_in_notebook(show_table=True, show_all=False)